# Solving systems of ODEs

So far we have only considered ODEs with a single solution component, often called scalar ODEs.
Many interesting processes can be described
by systems of ODEs, i.e., multiple ODEs where the right hand side of one equation depends on the solution of the others. Such equation
systems are also referred to as vector ODEs. One simple example is

$$
\begin{alignat*}{2}
u' &= v, \quad &&u(0) = 1\\
v' &= -u, \quad &&v(0) = 0.
\end{alignat*}
$$

The solution of this system is $u=\cos t, v=\sin t$, which can easily be verified by insterting the solution into the equations
and initial conditions. For more general cases, it is usually even more difficult to find analytical solutions of ODE systems
than of scalar ODEs, and numerical methods are usually required. In this chapter we will extend the solvers introduced in
Chapter 2 to be able to solve systems of ODEs. We shall see that such an extension requires relatively small
modifications of the code.

We want to develop general software that can be applied to any vector ODE or scalar ODE, and for this purpose it is
useful to introduce general mathematical notation. We have $n$ unknowns

$$
u^{(0)}(t), u^{(1)}(t), \ldots, u^{(n-1)}(t)
$$

in a system of $n$ ODEs:

$$
\begin{align*}
{d\over dt}u^{(0)} &= f^{(0)}(u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}, t),\\
{d\over dt}u^{(1)} &= f^{(1)}(u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}, t),\\
\vdots &= \vdots\\
{d\over dt}u^{(n-1)} &= f^{(n-1)}(u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}, t).
\end{align*}
$$

To simplify the notation (and later the implementation), we collect both the solutions $u^{(i)}(t)$
and right-hand side functions $f^{(i)}$ in vectors;

$$
u = (u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}),
$$

and

$$
f = (f^{(0)}, f^{(1)}, \ldots, f^{(n-1)}).
$$

Note that $f$ is now a vector-valued function. It takes $n+1$ input arguments ($t$ and the $n$ components of $u$) and returns
a vector of $n$ values.
The ODE system can now be written

$$
u' = f(u, t),\quad u(0) = u_0
$$

where $u$ and $f$ are vectors and $u_0$ is a vector of initial conditions. We see that we use exactly the
same notation as for scalar ODEs, and whether we solve a scalar or system of ODEs is determined by how we define $f$ and the initial
condition $u_0$. This general notation is completely standard in text books on ODEs, and we can easily make the Python
implementation just as general.

# An `ODESolver` class for systems of ODEs
The `ODESolver` class above was written for a scalar ODE. We now want to make it work for a system
$u'=f$, $u(0)=U_0$, where $u$, $f$ and $U_0$ are vectors (arrays). To identify how the code needs to be changed, let us start with
the simplest method. Applying the forward Euler method to a system of ODEs yields an update formula that
looks exactly as for the scalar case, but where all the terms are vectors:

$$
\underbrace{u_{k+1}}_{\mbox{vector}} =
\underbrace{u_k}_{\mbox{vector}} +
\Delta t\, \underbrace{f(u_k, t_k)}_{\mbox{vector}} .
$$

We could also write this formula in terms of the individual components, as in

$$
u^{(i)}_{k+1} = u^{(i)}_{k} + \Delta t f^{(i)}(u_{k},t_k), \mbox{ for } i = 0,\ldots , {n-1},
$$

but the compact vector notation is much easier to read. Fortunately, the way we write the vector
version of the formula is also how NumPy arrays are used in calculations. The
Python code for the formula above may therefore look idential to the version for scalar ODEs;

In [1]:
u[k+1] = u[k] + dt*f(u[k], t)

with the important difference that both `u[k]` and `u[k+1]` are now arrays.
Since these are arrays, the solution `u` must be a
two-dimensional array, and `u[k],u[k+1]`, etc. are the rows of this array.
The function `f` expects an array as its first argument, and must return a one-dimensional array,
containing all the right-hand sides $f^{(0)},\ldots,f^{(n-1)}$. To get a better
feel for how these arrays look and how they are used,
we may compare the array holding the solution of a scalar ODE to that of a system of two ODEs.
For the scalar equation, both `t` and `u` are one-dimensional NumPy
arrays, and indexing into `u` gives us numbers, representing the solution at each time step:

        t = [0.  0.4 0.8 1.2 (...) ]
        
        u = [ 1.0 1.4  1.96 2.744  (...)]
        
        u[0] = 1.0
        u[1] = 1.4
        
        (...)


In the case of a system of two ODEs, `t` is still a one-dimensional array, but the solution array `u` is
now two-dimensional, with one column for each solution component. Indexing into it
yields one-dimensional arrays of length two, which are the two solution components
at each time step:

        u = [[1.0 0.8][1.4 1.1] [1.9 2.7] (...)]
        
        u[0] = [1.0 0.8]
        u[1] = [1.4 1.1]
        
        (...)


The similarity of the generic notation for vector and scalar ODEs, and the
convenient algebra of NumPy arrays, indicate that the solver
implementation for scalar and system ODEs can also be very similar. This is indeed true,
and the `ODESolver` class from the previous chapter can be made to work for ODE
systems by a few minor modifactions:
 * Ensure that `f(u,t)` always returns an array.

 * Inspect $U_0$ to see if it is a single number or a list/array/tuple and
   make the `u` either a one-dimensional or two-dimensional  array

If these two items are handled and initialized correctly, the rest of the code from
Chapter 2 will in fact work with no modifications.
The extended superclass implementation may look like:

In [2]:
class ODESolver:
    def __init__(self, f):
        # Wrap user's f in a new function that always
        # converts list/tuple to array (or let array be array)
        self.f = lambda u, t: np.asarray(f(u, t), float)

    def set_initial_condition(self, U0):
        if isinstance(U0, (float,int)):  # scalar ODE
            self.neq = 1                 # no of equations
            U0 = float(U0)
        else:                            # system of ODEs
            U0 = np.asarray(U0)
            self.neq = U0.size           # no of equations
        self.U0 = U0

    def solve(self, time_points):
        self.t = np.asarray(time_points)
        N = len(self.t)
        if self.neq == 1:  # scalar ODEs
            self.u = np.zeros(N)
        else:              # systems of ODEs
            self.u = np.zeros((N,self.neq))

        # Assume that self.t[0] corresponds to self.U0
        self.u[0] = self.U0

        # Time loop
        for n in range(N-1):
            self.n = n
            self.u[n+1] = self.advance()
        return self.u, self.t

It is worth commenting on some parts of this code. First, the constructor looks
almost identical to the scalar case, but we use a lambda function and
`np.asarray` to convert any `f` that returns a list or tuple to a function
returning a NumPy array. This modification is not strictly
needed, since we could just assume that the user implements `f` to return
an array, but it makes the class more robust and flexible. We have also included
tests in the `set_initial_condition` method, to check if `U0` is a single
number (`float`) or a NumPy array, and define the attribute `self.neq` to
hold the number of equations.
The final modification is found in the method `solve`, where
the `self.neq` attribute is inspected and `u` is
initialized to a one- or two-dimensional array of the correct size. The
actual for-loop, as well as the implementation of the `advance` method in the
subclasses, can be left unchanged.

### Example: ODE model for throwing a ball.

To demonstrate the use of
the extended `ODESolver` hierarchy, let us derive and solve a system of ODEs
describing the trajectory of a ball.
We first define $x(t),y(t)$ to be the position of the ball, $v_x$ and $v_y$ the velocity components,
and $a_x,a_y$ the acceleration components. From the definition of velocity and acceleration,
we have $v_x = dx/dt, v_y = dy/dt, a_x = dv_x/dt$, and $a_y = dv_y/dt$.
If we neglect air resistance there are no forces acting on the ball in the $x$-direction,
so from Newton's second law we have $a_x = 0$. In the $y$-direction the acceleration
must be equal to the acceleration of gravity, which yields $a_y = -g$.
In terms of the velocities, we have

$$
\begin{align*}
a_x &= 0 \quad \Rightarrow \quad \frac{dv_x}{dt} = 0, \\
a_y &= -g \quad \Rightarrow \quad \frac{dv_y}{dt} = -g\mbox{\ } ,
\end{align*}
$$

and the complete ODE system can be written as

<!-- Equation labels as ordinary links -->
<div id="vx"></div>

$$
\begin{equation}
{dx\over dt} = v_x, \label{vx} \tag{1} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="ax"></div>

$$
\begin{equation} 
{dv_x\over dt} = 0,\label{ax} \tag{2} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="vy"></div>

$$
\begin{equation} 
{dy\over dt} = v_y,\label{vy} \tag{3} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="ay"></div>

$$
\begin{equation} 
{dv_y\over dt} = -g.\label{ay} \tag{4}
\end{equation}
$$

To solve the system we need to define initial conditions for all four unknowns,
i.e., we need to know the initial position and velocity of the ball.

A closer inspection of the system ([1](#vx))-([4](#ay)) will reveal that although
this is a coupled system of ODEs, the coupling is in fact quite weak and the system is
easy to solve analytically. There is essentially a one-way coupling between equations
([2](#ax)) and ([1](#vx)), the same between ([4](#ay)) and ([3](#vy)), and no other coupling
between the equations. We can easily solve ([2](#ax)) to conclude that $v_x$ is a constant,
and inserting a constant on the right hand side of ([1](#vx)) yields that $x$ must be a linear
function of $t$. Similarly, we can solve ([4](#ay)) to find that $v_y$ is a linear function, and then
insert this into ([3](#vy)) to find that $y$ is a quadratic function of $t$. The functions
$x(t)$ and $y(t)$ will contain four unknown coefficients that must be determined from the
initial conditions.

Although the analytical solution is available, we want to use the `ODESolver`
class hierarchy presented above to solve this system. The first step is then to
implement the right hand side as a Python function:

In [3]:
def f(u, t):
    x, vx, y, vy = u
    g = 9.81
    return [vx, 0, vy, -g]

We see that the function here returns a list, but this will automatically
be converted to an array by the solver class' constructor,
as mentioned above. The main program is not very different
from the examples of the previous chapter, except that we need to define an
initial condition with four components:

In [4]:
%matplotlib inline

from ODESolver import ForwardEuler
import numpy as np
import matplotlib.pyplot as plt

# Initial condition, start at the origin:
x = 0; y = 0
# velocity magnitude and angle:
v0 = 5; theta = 80*np.pi/180
vx = v0*np.cos(theta); vy = v0*np.sin(theta)

U0 = [x, vx, y, vy]

solver= ForwardEuler(f)
solver.set_initial_condition(U0)
time_points = np.linspace(0, 1.0, 101)
u, t = solver.solve(time_points)
# u is an array of [x,vx,y,vy] arrays, plot y vs x:
x = u[:,0];  y = u[:,2]

plt.plot(x, y)
plt.show()

Notice that since `u` is a two-dimensional array, we use array slicing to extract and plot the individual components.
A call like `plt.plot(t,u)` will also work, but it will plot all the solution
components in the same window, which for this particular model is not very useful.
A very useful exercise is to extend this code to plot the analytical solution of the system in the
same window as the numerical solution. The system can be solved as outlined above, and the
unknown coefficients in the solution formulas can be determined from the given initial conditions.
With the chosen number of time steps there will be a visible difference between the numerical
solution and the analytical solution, but this can easily be removed
by reducing the time step or choosing a more accurate solver.